In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
from gensim.models import FastText

import nltk
nltk.download("stopwords")
#--------#

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import re

from scipy.spatial.distance import cosine

In [3]:
model = FastText.load('drive/My Drive/Colab Notebooks/hackaton/fasttext.model')

In [4]:
data= pd.read_csv('drive/My Drive/Colab Notebooks/hackaton/relations_full.csv', sep=';')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (37) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
data = data[['id_ученика','пол','возраст','Наименование']].drop_duplicates() #Наименование_услуги или Наименование (?)
data.columns = ['id', 'sex', 'age', 'name'] 

In [11]:
embeddings = model.wv[data['name']]

In [13]:
data['embedding']=embeddings.tolist()

In [45]:
users = data[['id', 'embedding']]

In [48]:
users.head(1)

,id,embedding
0,25969.0,"[0.23481345176696777, -0.010903395712375641, -..."


In [105]:
users_emb = users.groupby('id', as_index=False)['embedding'].sum()

In [108]:
users_emb.head()

,id,embedding
0,1.0,"[0.23481345176696777, -0.010903395712375641, -..."
1,2.0,"[0.23481345176696777, -0.010903395712375641, -..."
2,3.0,"[0.23481345176696777, -0.010903395712375641, -..."
3,4.0,"[0.23481345176696777, -0.010903395712375641, -..."
4,5.0,"[0.23481345176696777, -0.010903395712375641, -..."


In [157]:
users_emb.to_csv('drive/My Drive/Colab Notebooks/hackaton/user_emb.csv')

In [70]:
events = pd.read_csv('drive/My Drive/Colab Notebooks/hackaton/events.csv')

In [71]:
events=events.fillna('')

In [92]:
#Create lemmatizer and stopwords list
mystem = Mystem() 
stemmer = SnowballStemmer('russian')
russian_stopwords = stopwords.words("russian")

#Preprocess function
def preprocess_text(text):
    
    tokens = re.sub(r'[^A-Za-zА-Яа-яёЁ0-9-\s]',r'',str(text).lower())
    #tokens = mystem.lemmatize(text.lower())
    tokens = tokens.split(' ')
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    #text = " ".join(tokens)
    
    tokens = [stemmer.stem(word) for word in tokens] #??????????

    return tokens

#Preprocess function
def preprocess_text2(text):
    
    tokens = re.sub(r'[^A-Za-zА-Яа-яёЁ0-9-\s]',r'',str(text).lower())
    #tokens = mystem.lemmatize(text.lower())
    #tokens = tokens.split(' ')
    #tokens = [token for token in tokens if token not in russian_stopwords\
              #and token != " " \
              #and token.strip() not in punctuation]
    #text = " ".join(tokens)
    
    #tokens = [stemmer.stem(word) for word in tokens] #??????????

    return tokens

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [93]:
events['text']=events['Название мероприятия']+' '+events['Наименование учреждения']+' '+events['Направленность мероприятия']+' '+events['Краткое описание']

In [94]:
text_new=events["text"].apply(preprocess_text2)
#text_new =events["text"].apply(preprocess_text)
events['text_new'] = text_new


In [98]:
events.head(2)

,id,Название мероприятия,Статус,Наименование учреждения,Тип учреждения,Подразделение,Стоимость (полная),Тип мероприятия,Направленность мероприятия,Мероприятие относится к ежегодным праздникам и общегородским мероприятиям,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Доступность мероприятия для лиц с ОВЗ,Тип ОВЗ,Возрастной ценз участников мероприятия,Возрастная категория,Целевая аудитория,text,text_new
0,5,Фестивлаь современных молодежных уличных культ...,Завершено,ГБУК г. Москвы «Клуб «Современник»,Дома культуры,,бесплатное,фестиваль,культурно-досуговое,,,13.06.2020 16:00,,13.06.2020 20:00,,Клуб Современник проведет фестиваль современны...,Нет,,,,недоступно,,0+,от 20 до 40,,Фестивлаь современных молодежных уличных культ...,фестивлаь современных молодежных уличных культ...
1,12,"Танцевальный вечер ""Ретро"" в честь Дня Победы ...",Завершено,ГБУК г.Москвы «ДК «Нагорный»,Дома культуры,,бесплатное,"танцевальный вечер, бал",патриотическое,праздник,9 мая. День Победы,09.05.2020 14:30,,09.05.2020 17:30,,"Танцевальный вечер ""Ретро"" в честь Дня Победы",Нет,"Электролитный пр. 3, корпус 1",Южный административный округ,Нагорный,недоступно,,16+,от 40 до 85,"работающая молодежь, взрослые, пенсионеры","Танцевальный вечер ""Ретро"" в честь Дня Победы ...",танцевальный вечер ретро в честь дня победы ва...


In [97]:
embeddings_event = model.wv[events['text_new']]

In [99]:
events['embedding']=embeddings_event.tolist()

In [168]:
def compare_distance(x, user_emb):
  return cosine(x, user_emb)

events['distances'] = events['embedding'].apply((lambda x: compare_distance(x, users_emb['embedding'][1000])))

In [172]:
events.sort_values('distances', ascending=True)[['id', 'Название мероприятия', 'Дата начала мероприятия', 'Наименование учреждения', 'distances']]\
.drop_duplicates(subset=['Название мероприятия']).drop_duplicates(subset=['Наименование учреждения']).head()#.to_list()

,id,Название мероприятия,Дата начала мероприятия,Наименование учреждения,distances
35072,7650,Friday Point English,28.02.2020 19:00,ГАУК г. Москвы «Культурный центр ЗИЛ»,0.231875
40795,13373,PEOPLE&THEIR STORIES,23.02.2020 14:00,ГБУК г. Москвы «КЦ «Братеево»,0.758046
23339,8928,Занятие по английскому языку от студии «Family...,06.07.2020,ГБУК г. Москвы «Московский продюсерский центр»,0.816008
4719,4735,The Little House,05.06.2020 17:00,ГБУК г. Москвы «ДК «Дружба»,0.866624
3177,4199,"Концерт Трио ""Maestro Classic""",01.02.2020,ГБУК г.Москвы «ЦБС ЮВАО»,0.888833
